# 自定义输出
- 输出函数参数
- 输出json
- 输出List
- 输出日期

In [9]:

#讲笑话机器人：希望每次根据指令，可以输出一个这样的笑话(小明是怎么死的，笨死的)

from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel,Field, validator
from typing import List
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")

chat = ChatOpenAI (
    model="gpt-3.5-turbo-16k",
    temperature=0,
    max_tokens=1000,
    openai_api_base=openai_api_base,
    openai_api_key=openai_api_key,  #
)

#定义个数据模型，用来描述最终的实例结构

class Joke(BaseModel):
    setup:str = Field(description="设置笑话的问题")
    punchline:str  = Field(description="回答笑话的答案")

    #验证问题是否符合要求 
    @validator("setup")
    def question_mark(cls,field):
        if field[-1] != "？":
            raise ValueError('不符合预期的问题格式！')
        return field
    
#将Joke数据模型传入 
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template = "回答用户的输入.\n{format_instructions}\n{query}\n",
    input_variables = ["query"],
    partial_variables = {"format_instructions": parser.get_format_instructions()}
)

prompt_and_model = prompt | chat
out_put = prompt_and_model.invoke({"query":"给我讲一个笑话"})
print("out_put", out_put)
parser.invoke(out_put)

out_put content='{\n  "setup": "为什么鸟儿不会玩扑克牌？",\n  "punchline": "因为它们会把鬼牌都当作鸟牌！"\n}' response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 248, 'total_tokens': 307, 'pre_token_count': 1000, 'pre_total': 25, 'adjust_total': 20, 'final_total': 5}, 'model_name': 'gpt-3.5-turbo-16k', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a0375521-da44-41d9-bcc4-48252a99d17d-0'


Joke(setup='为什么鸟儿不会玩扑克牌？', punchline='因为它们会把鬼牌都当作鸟牌！')

# LLM的输出格式化成python list形式， 类似['a', 'b','c']

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from typing import List
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")

chat = ChatOpenAI (
    model="gpt-3.5-turbo-16k",
    temperature=0,
    max_tokens=1000,
    openai_api_base=openai_api_base,
    openai_api_key=openai_api_key,  #
)


#将Joke数据模型传入 
parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template = "列出5个{subject}.\n{format_instructions}\n",
    input_variables = ["subject"],
    partial_variables = {"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format(subject="常见中国人的名字")
output=chat(_input)
print(output)
#格式化
parser.parse(output)

